# FRA(선도금리계약)

- 미래의 단기 금리 리스크를 헷지하기 위해 고안된 파생상품
- 금리 상승리스크 투자자 -> FRA매수로 리스크 헷지
- 금리 하락리스크 투자자 -> FRA매도로 리스크 헷지
- 금리 방향성 특정 가능한 경우 네이키드 FRA포지션으로 방향성 매매

# SWAP_CURVE 모듈 불러오기

In [1]:
import datetime
import QuantLib as ql
from SWAP_CURVE import GET_QUOTE, SWAP_CURVE

---------------------------------------------------------

- init = 클래스 내부의 속성 초기화 역할, 입력변수 있다면 그 변수를 내부화
- 평가일자, 정산일자, 만기일자, 포지션, FRA금리, 명목금액 외부로부터 내부화

In [2]:
class FRA():
    def __init__(self, todays_date, value_date, maturity_date, position, fra_rate, notional):
        
        # Initial Setup 1 : Date & Curve
        self.date = todays_date
        self.curve = self.CURVE(self.date)
        
        # Initial Setup 2 : Instruments Info
        self.value_date = ql.Date(value_date.day, value_date.month, value_date.year)
        self.maturity_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year)
        
        if position =='Long':
            self.position = ql.Position.Long
        else:
            self.position = ql.Position.Short
            
        self.fra_rate = fra_rate
        self.notional = notional
        
        # Pricing Results ( 클래스 내에서 구현한 다른 메서드를 속성 처럼 )
        self.npv = self.PRICING(self.curve)
        self.delta = self.DELTA()
        self.theta = self.THETA()
    def CURVE(self, date):
        return SWAP_CURVE(date, GET_QUOTE(date)) 
    def PRICING(self, curve):
        # Yield Term-structure
        curve_handle = ql.YieldTermStructureHandle(curve)
    
        # Floating rate
        libor = ql.USDLibor(ql.Period(3, ql.Months), curve_handle)
    
        # Pricing FRA
        fra = ql.ForwardRateAgreement(self.value_date,
                                     self.maturity_date,
                                     self.position,
                                     self.fra_rate,
                                     self.notional,
                                     libor,
                                     curve_handle)
        # FRA price
        npv = fra.NPV()
    
        return npv
    def DELTA(self):
        curve_handle = ql.YieldTermStructureHandle(self.curve)
    
        # 1bp
        basis_point = 0.0001
    
        # FRA price when 1bp up
        up_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(basis_point)))
        #up_curve_handle = ql.YieldTermStructureHandle(up_curve)
        up_fra = self.PRICING(up_curve)
    
        # FRA price when 1bp down
        down_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(-basis_point)))
        #down_curve_handle = ql.YieldTermStructureHandle(down_curve)
        down_fra = self.PRICING(down_curve)
    
        # DV01
        dv01 = (up_fra - down_fra) / 2
    
        return dv01
    def THETA(self):
        price_t0 = self.PRICING(self.CURVE(self.date))
        price_t1 = self.PRICING(self.CURVE(self.date + datetime.timedelta(days=1)))
    
        return price_t1 - price_t0

curve = 기존의 스왑 커브 모듈을 재활용하여 클래스 내부에서 스왑 커브를 만들어주는 역할

- 델타: 기초자산의 가격이 한 단위 변했을 때 파생상품 가격이 얼마나 변하는가
- 커브가 1bp 상승했을 때/하락했을 때

# TEST

In [3]:
if __name__ == "__main__":
    
    # Today's Date
    todays_date = datetime.date(2020, 10, 9)
    
    # FRA Instrument Setup
    value_date = datetime.date(2021, 1, 9)
    maturity_date = datetime.date(2021, 4, 9)
    position = 'Long'
    fra_rate = 0.0022
    notional = 10000000
    
    # Build FRA object
    fra = FRA(todays_date,
             value_date,
             maturity_date,
             position,
             fra_rate,
             notional)
    
    # Print Results
    print("Price = {}".format(round(fra.npv, 4)))
    print("Price = {}".format(round(fra.delta, 4)))
    print("Price = {}".format(round(fra.theta, 4)))

Price = 529.9903
Price = 249.8317
Price = -0.6453
